<a href="https://colab.research.google.com/github/gwnwytt/GwenWyattDTSC3020Fall2025/blob/main/gw0183_assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [1]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [2]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [5]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
  tables = pd.read_html(html)
  raw_df = None
  for t in tables:
    if t.shape[1] >= 3:
      raw_df = t
      break
  if raw_df is None:
    raw_df = tables[0]

  if isinstance(raw_df.columns, pd.MultiIndex):
    raw_df.columns = [' '.join([str(x) for x in tup if str(x) != 'nan']).strip()
                          for tup in raw_df.columns.values]
  else:
    raw_df.columns = [str(c).strip() for c in raw_df.columns]

  return raw_df

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
  text_col = next ((c for c in df.columns if df[c].dtype == 'object'), df.columns[0])

  def numeric_score(series):
    s = series.astype(str).str.replace(r"[^\d]", "", regex=True)
    return (s != "").sum()
  num_col = max(df.columns, key=lambda c: numeric_score(df[c]))

  def pick_alpha(col_list, length):
    for c in col_list:
      vals = df[c].dropna().astype(str).str.strip()
      if(vals.str.len() == length).mean() > 0.5:
        return c
    return None

  alpha2_col = pick_alpha(df.columns, 2)
  alpha3_col = pick_alpha(df.columns, 3)

  df.columns = ["Country", "Alpha-2", "Alpha-3", "Numeric"]

  df["Country"] = (df["Country"].astype(str)
                   .str.replace(r"\[.*?\]", "", regex=True)
                   .str.strip())
  df["Alpha-2"] = (df["Alpha-2"].astype(str)
                   .str.replace(r"\[.*?\]", "", regex=True)
                   .str.strip()
                   .str.upper()
                   .str[:2])
  df["Alpha-3"] = (df["Alpha-3"].astype(str)
                   .str.replace(r"\[.*?\]", "", regex=True)
                   .str.strip()
                   .str.upper()
                   .str[:3])
  df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

  return df

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
  df_sorted = df.sort_values(by="Numeric", ascending=False, na_position="last")
  return df_sorted.head(top).reset_index(drop=True)


In [6]:
# Q1 — Write your answer here
url = "https://www.iban.com/country-codes"

r = requests.get(url, headers=HEADERS, timeout=20)
r.raise_for_status()
html = r.text
soup = BeautifulSoup(html, "lxml")

df = q1_clean(q1_read_table(html))
top15 = q1_sort_top(df)
display(top15)

df.to_csv("data_q1.csv", index=False)


/tmp/ipython-input-707337564.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


,Country,Alpha-2,Alpha-3,Numeric
0,Zambia,ZM,ZMB,894
1,Yemen,YE,YEM,887
2,Samoa,WS,WSM,882
3,Wallis and Futuna,WF,WLF,876
4,Venezuela (Bolivarian Republic of),VE,VEN,862
5,Uzbekistan,UZ,UZB,860
6,Uruguay,UY,URY,858
7,Burkina Faso,BF,BFA,854
8,Virgin Islands (U.S.),VI,VIR,850
9,United States of America (the),US,USA,840


# Country Codes Extraction (IBAN.com)

**URL:** [https://www.iban.com/country-codes](https://www.iban.com/country-codes)

## Steps
1. Downloaded HTML using `requests`.
2. Parsed the first `<table>` with ≥3 columns using `pandas.read_html()`.
3. Cleaned columns:
   - Trimmed whitespace.
   - Uppercased Alpha-2 and Alpha-3 codes.
   - Converted Numeric codes to nullable integers.
4. Sorted by `Numeric` descending and displayed the Top 15.
5. Saved the cleaned data to `data_q1.csv`.

## Output Files
- **data_q1.csv** — cleaned country code table  
- **notebook.ipynb** — code execution steps  
- **Top 15 printed in output**

## Limitation
- The column detection is heuristic (based on string length for Alpha-2/Alpha-3).  
  If the source table changes column order or adds notes, detection may fail.


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [12]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    items = []
    stories = soup.select(".athing")

    for story in stories:
      rank_tag = story.select_one(".rank")
      title_tag = story.select_one(".titleline a")

      rank = rank_tag.text.replace(".", "") if rank_tag else ""
      title = title_tag.text.replace(".", "") if title_tag else ""
      link = title_tag['href'] if title_tag and title_tag.has_attr("href") else ""

      sub = story.find_next_sibling("tr")
      subtext = sub.select_one(".subtext") if sub else None

      if subtext:
        points_tag = subtext.select_one(".score")
        user_tag = subtext.select_one(".hnuser")

        comment_tag = None
        for a in subtext.find_all("a"):
          if "comment" in a.text.lower():
            comment_tag = a
            break

        points = points_tag.text if points_tag else ""
        user = user_tag.text if user_tag else ""
        comments = comment_tag.text if comment_tag else ""

      else:
        points, user, comments, "", "", ""

      items.append({
          "rank": rank,
          "title": title,
          "link": link,
          "points": points,
          "comments": comments,
          "user": user
      })

    df = pd.DataFrame(items)
    return df

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    for col in ["rank", "points", "comments"]:
      df[col] = (df[col].astype(str)
                  .str.extract(r"(\d+)", expand=False)
                  .fillna(0)
                  .astype(int))
    for col in ["title", "link", "user"]:
      df[col] = df[col].fillna("").replace("", "(missing)")

    return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    df_sorted = df.sort_values(by='rank', ascending=True, na_position="last")
    return df_sorted.head(top).reset_index(drop=True)


In [13]:
# Q2 — Write your answer here
url = "https://news.ycombinator.com/"

r = requests.get(url, headers=HEADERS, timeout=20)
r.raise_for_status()
html = r.text
soup = BeautifulSoup(html, "lxml")

df = q2_clean(q2_parse_items(html))
top15 = q2_sort_top(df)
display(top15)

df.to_csv("data_q1.csv", index=False)




,rank,title,link,points,comments,user
0,1,Myna: Monospace typeface designed for symbol-h...,https://github.com/sayyadirfanali/Myna,108,45,birdculture
1,2,Ruby Solved My Problem,https://newsletter.masilotti.com/p/ruby-alread...,95,26,joemasilotti
2,3,How did I get here?,https://how-did-i-get-here.net/,41,10,zachlatta
3,4,Ribir: Non-intrusive GUI framework for Rust/WASM,https://github.com/RibirX/Ribir,23,2,adamnemecek
4,5,"YouTube Removes Windows 11 Bypass Tutorials, C...",https://news.itsfoss.com/youtube-removes-windo...,71,13,WaitWaitWha
5,6,Venn Diagram for 7 Sets,https://moebio.com/research/sevensets/,57,12,bramadityaw
6,7,I Love OCaml,https://mccd.space/posts/ocaml-the-worlds-best/,252,160,art-w
7,8,James Watson has died,https://www.nytimes.com/2025/11/07/science/jam...,131,32,granzymes
8,9,Leaving Meta and PyTorch,https://soumith.ch/blog/2025-11-06-leaving-met...,652,158,saikatsg
9,10,VLC's Jean-Baptiste Kempf Receives the Europea...,https://fsfe.org/news/2025/news-20251107-01.en...,68,5,kirschner
